Для сигналізації про аварію встановлено два незалежно працюючих сигналізатори. Ймовірність того, що
при аварії спрацює перший сигналізатор дорівнює 0,95, ця ймовірність для другого сигналізатора дорівнює 0,9.
Знайти ймовірність того, що при аварії спрацює:
а) хоча б один сигналізатор;
б) тільки один сигналізатор.

Вирішення:


а)  Подія А_1 - включення першого сигналізатора;
    Подія A_2 - включення другого сигналізатора;
    Події А_1 і A_2 сумісні, тому використовується теорема додавання:

In [1]:
P_A_1 = (95/100) + (90/100) - (855/1000)
print(f"Дана подія відбудеться з вірогідністю {round(P_A_1 * 100, 2)} відсотків")

Дана подія відбудеться з вірогідністю 99.5 відсотків


б)  Один і тільки один сигналізатор може включитися у двох випадках.
    
Перший випадок:    
    Подія А_1 - включення першого сигналізатора;
    Подія A_2 - невключення другого сигналізатора;
    Події А_1 і A_2 незалежні, тому використовується теорема множення:

In [2]:
P_A_2_1 = (95/100) * (10/100)
print(f"З вірогідністю {round(P_A_2_1 * 100, 2)} відсотків включиться лише перший сигналізатор")

З вірогідністю 9.5 відсотків включиться лише перший сигналізатор


Другий випадок:    
    Подія А_3 - невключення першого сигналізатора;
    Подія A_4 - включення другого сигналізатора;
    Події А_3 і A_4 незалежні, тому використовується теорема множення:

In [3]:
P_A_2_2 = (5/100) * (90/100)
print(f"З вірогідністю {round(P_A_2_2 * 100, 2)} відсотків включиться лише другий сигналізатор")

З вірогідністю 4.5 відсотків включиться лише другий сигналізатор


Обидва даних випадки являються сумісними, тому для них використовується теорема додавання:

In [2]:
P_A_2_3 = (95/100) * (10/100) + (5/100) * (90/100)
print(f"З вірогідністю {round(P_A_2_3 * 100, 2)} % включиться лише один сигналізатор")

З вірогідністю 14.0 % включиться лише один сигналізатор
